## Read in Data

In [1]:
import numpy as np
import csv

# Predict via the median number of plays.

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'global_median.csv'

artist_file = "artists.csv"
profile_file = "profiles.csv"

# Load the training data.
train_data = {}
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = int(row[2])
    
        if not user in train_data:
            train_data[user] = {}
        
        train_data[user][artist] = plays


artist_data = {}        
with open(artist_file, 'r') as artist_fh:
    artist_csv = csv.reader(artist_fh, delimiter=',', quotechar='"')
    next(artist_csv, None)
    for row in artist_csv:
        artist   = row[0]
        name = row[1]
          
        artist_data[artist] = name



In [2]:
import pandas as pd
train_data = pd.read_csv("train.csv")
artist_data=pd.read_csv("artists.csv")
profile_data = pd.read_csv("profiles.csv")

In [3]:
profile_data.head(10)

,user,sex,age,country
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25,Sweden
1,5909125332c108365a26ccf0ee62636eee08215c,m,29,Iceland
2,d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30,United States
3,63268cce0d68127729890c1691f62d5be5abd87c,m,21,Germany
4,02871cd952d607ba69b64e2e107773012c708113,m,24,Netherlands
5,0938eb3d1b449b480c4e2431c457f6ead7063a34,m,22,United States
6,e4c6b36e65db3d48474dd538fe74d2dbb5a2e79e,f,NaN,United States
7,b97479f9a563a5c43b423a976f51fd509e1ec5ba,f,NaN,Poland
8,3bb020df0ff376dfdded4d5e63e2d35a50b3c535,m,NaN,United States
9,f3fb86c0f024f640cae3fb479f3a27e0dd499891,NaN,16,Ukraine


In [4]:
profile_data.describe()

,age
count,188444.000000
mean,24.517400
std,21.853296
min,-1337.000000
25%,20.000000
50%,23.000000
75%,27.000000
max,1002.000000


In [5]:
profile_data.isnull().sum()

user           0
sex        19535
age        44842
country        0
dtype: int64

In [29]:
profile_data.shape

(233286, 5)

In [9]:
countrydict ={}
i=0
for country in profile_data.country.unique():
    countrydict[country]=i
    i=i+1
    
profile_data['countryidx'] = profile_data['country'].apply(lambda x: countrydict[x])
profile_data.head()

,user,sex,age,country,countryidx
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25,Sweden,0
1,5909125332c108365a26ccf0ee62636eee08215c,m,29,Iceland,1
2,d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30,United States,2
3,63268cce0d68127729890c1691f62d5be5abd87c,m,21,Germany,3
4,02871cd952d607ba69b64e2e107773012c708113,m,24,Netherlands,4


In [10]:
train_data.head()

,user,artist,plays
0,eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,5a8e07d5-d932-4484-a7f7-e700793a9c94,554
1,44ce793a6cd9d20f13f4a576a818ef983314bb5d,a3a92047-be1c-4f3e-8960-c4f8570984df,81
2,da9cf3f557161d54b76f24db64be9cc76db008e3,eeb1195b-f213-4ce1-b28c-8565211f8e43,708
3,8fa49ab25d425edcf05d44bfc1d5aea895287d81,a1419808-65d3-4d40-998c-1a0bac65eabc,265
4,b85fcaef67d2669cd99b334b5e8c8705263db2cf,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,220


In [31]:
train_data.shape

(4154804, 3)

In [11]:
tmp_data = pd.merge(train_data,profile_data,on='user')
main_data = pd.merge(tmp_data,artist_data,on='artist')

In [12]:
main_data.head()

,user,artist,plays,sex,age,country,countryidx,name
0,eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,5a8e07d5-d932-4484-a7f7-e700793a9c94,554,m,25,Sweden,0,Robyn
1,0ff4166398f035b5fcb8824cc16c8daeb4643911,5a8e07d5-d932-4484-a7f7-e700793a9c94,169,f,18,United Kingdom,16,Robyn
2,b3f9fa56429c3b7fd348c471452e65747ba9ed50,5a8e07d5-d932-4484-a7f7-e700793a9c94,292,m,23,United Kingdom,16,Robyn
3,0ffff52af79555e8fe72289c429b2fdfc8ea684b,5a8e07d5-d932-4484-a7f7-e700793a9c94,92,m,26,Germany,3,Robyn
4,985253be0dc82ffa15a0ad006d0284aa4b7d1e3d,5a8e07d5-d932-4484-a7f7-e700793a9c94,159,m,19,Sweden,0,Robyn


In [14]:
train_df = main_data[['countryidx']]
y_df= main_data[['plays']]
train_set = train_df.as_matrix()
y_set = y_df.as_matrix()

In [22]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(train_df)
a= enc.transform(train_df).toarray()


TypeError: 'tuple' object is not callable

In [30]:
len(a)

4154804

In [22]:
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(train_set.shape[0]), train_size=0.1,random_state=123)
mask=np.ones(train_set.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

X_train = train_set[mask]
Y_train = y_set[mask]

array([['Sweden', 'Robyn'],
       ['Germany', 'Robyn'],
       ['Germany', 'Robyn'],
       ..., 
       ['Poland', 'Fabolous'],
       ['Germany', 'Fabolous'],
       ['Costa Rica', 'Fabolous']], dtype=object)

In [19]:
from sklearn.cluster import KMeans



array([['eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03',
        '5a8e07d5-d932-4484-a7f7-e700793a9c94', 554, ..., 25.0, 'Sweden',
        'Robyn'],
       ['0ff4166398f035b5fcb8824cc16c8daeb4643911',
        '5a8e07d5-d932-4484-a7f7-e700793a9c94', 169, ..., 18.0,
        'United Kingdom', 'Robyn'],
       ['b3f9fa56429c3b7fd348c471452e65747ba9ed50',
        '5a8e07d5-d932-4484-a7f7-e700793a9c94', 292, ..., 23.0,
        'United Kingdom', 'Robyn'],
       ..., 
       ['c84ba5695fc707c3c880154e8df58767943b7e93',
        'd2299099-e617-46dd-b013-ba0b4c72bc63', 37, ..., 23.0,
        'South Africa', 'Fabolous'],
       ['ecfb7d1e18dac370fec68309d918f4970cb686ad',
        'd2299099-e617-46dd-b013-ba0b4c72bc63', 31, ..., 20.0,
        'Russian Federation', 'Fabolous'],
       ['f1caee5b7d959bdcc1f0a10c1d03f6e506d85b17',
        'd2299099-e617-46dd-b013-ba0b4c72bc63', 220, ..., nan,
        'United States', 'Fabolous']], dtype=object)